參考:https://github.com/aws-samples/amazon-forecast-samples/blob/master/notebooks/basic/Tutorial/1.Importing_Your_Data.ipynb

In [55]:
import sys
import os
import json
import pandas as pd
%reload_ext autoreload
import boto3
import s3fs
import time

In [56]:
region = 'ap-northeast-1'
# BUCKET_NAME = ''

In [57]:
# Create API Session
session = boto3.Session(region_name=region)
forecast = session.client(service_name='forecast')
forecastquery = session.client(service_name='forecastquery')

In [58]:
# 建議一個funciton創IAM role
def get_or_create_iam_role( role_name ):

    iam = boto3.client("iam")

    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
              "Effect": "Allow",
              "Principal": {
                "Service": "forecast.amazonaws.com"
              },
              "Action": "sts:AssumeRole"
            }
        ]
    }

    try:
        create_role_response = iam.create_role(
            RoleName = role_name,
            AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
        )
        role_arn = create_role_response["Role"]["Arn"]
        print("Created", role_arn)
        
        print("Attaching policies...")
        iam.attach_role_policy(
            RoleName = role_name,
            PolicyArn = "arn:aws:iam::aws:policy/AmazonForecastFullAccess"
        )

        iam.attach_role_policy(
            RoleName=role_name,
            PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
        )

        print("Waiting for a minute to allow IAM role policy attachment to propagate")
        time.sleep(60)
    except iam.exceptions.EntityAlreadyExistsException:
        print("The role " + role_name + " exists, ignore to create it")
        role_arn = boto3.resource('iam').Role(role_name).arn

    print("Done.")
    return role_arn

In [65]:
# Create the role to provide to Amazon Forecast.
role_name = "ForecastNotebookRole-Basic"
print(f"Creating Role {role_name} ...")
role_arn = get_or_create_iam_role( role_name = role_name )


print(role_arn)


Creating Role ForecastNotebookRole-Basic ...
The role ForecastNotebookRole-Basic exists, ignore to create it
Done.
arn:aws:iam::070576557102:role/ForecastNotebookRole-Basic


In [60]:
# check you can communicate with Forecast API session
forecast.list_predictors()

{'Predictors': [],
 'ResponseMetadata': {'RequestId': '5f17966e-5b40-48f9-b5df-5f98fc1c9194',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 15 Mar 2024 03:30:51 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '17',
   'connection': 'keep-alive',
   'x-amzn-requestid': '5f17966e-5b40-48f9-b5df-5f98fc1c9194'},
  'RetryAttempts': 0}}